### Samuel Zeleke - CS344 - Homework 3

1. Compute the following values using the restaurant example.

    The *information gain* provided by using the price attribute as the root of the decision tree. Is it more or less 
valuable than the type or patrons attributes computed in class?

    ANSWER:
        Gain(Price) = 1 - Remainder(A)
                    = 1 - ((3/12) * B(1/3) + (2/12) * B(2/2) + (7/12) * B(3/7))
                    = 1 - (0.25 * -((1/3) * lg(1/3) + (2/3) * lg(2/3))
                           + (1/6) * -((2/2) * lg(2/2) + (0/2) * lg(0/2)) # 0*lg(0) is undefined. But, since the right
                                                                          # limit of x*lg(x) as x approaches 0 is 0, 
                                                                          # 0*lg(0) ~ 0
                           + (7/12) * -((3/7) * lg(3/7) + (4/7) * lg(4/7))
                      )
                    = 1 - (0.25 * -(-0.52832 -0.389975) + 
                            (1/6) * -(0) +   
                            (7/12) * -(-0.52388 - 0.46135) 
                          )
                    = 1 - 0.80429
                    = 0.1957
        - Since the gain for Patrons is greater than the gain for Price, asking about price is less valuable than asking 
        about Patrons.
        - Since the gain for Price is greater than the gain for Type, its more valuable than Type.

2. In class, we attempted to create by hand a neural network that computes the XOR function. If this was possible, see 
   if you can simplify the network we built. Consider relaxing the conventions of densely-connected, sequential layers. 
   If it was not possible, give a full explanation why it can’t be done.
   
   ANSWER:
        The XOR function is not linearly separable. So the function can not be represented by a perceptron network. 
        Instead, we should use to units (nodes) that learn to label (0, 1) and (1, 0) separately. Since XOR function has
        one output, we need another node in an output layer that aggregates the outputs of the nodes in the input layer.
        
        Since XOR(a, b) = OR(AND(a, ¬b), AND(¬a, b)), we need 2 nodes for the ANDs in the input layer and a node for the
        OR in the output layer.

3. Use Python/NumPy/Pandas/Keras to load and manipulate the Boston Housing Dataset as follows.

    a. Compute the dimensions of the data structures. Include code to print these values.
    
       ANSWER:

In [11]:
    
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

print("training data    dimension: %d, shape: %d x %d" %(train_data.ndim, train_data.shape[0], train_data.shape[1]))
print("training targets dimension: %d, shape: %d" %(train_targets.ndim, train_targets.shape[0]))
print("test data        dimension: %d, shape: %d x %d" %(test_data.ndim, test_data.shape[0], test_data.shape[1]))
print("test targets     dimension: %d, shape: %d" %(test_targets.ndim, test_targets.shape[0]))

training data    dimension: 2, shape: 404 x 13
training targets dimension: 1, shape: 404
test data        dimension: 2, shape: 102 x 13
test targets     dimension: 1, shape: 102


    b. Construct a suitable testing set, training set, and validation set for this data. Submit code to create these 
       datasets but do not include the datasets themselves.
       
       ANSWER:

In [12]:
import pandas
import math

def create_training_and_validation(taining, ratio = 0.3):
    
    training_set = pandas.DataSet()
    validation_set = pandas.DataSet()
    
    return training_set, validation_set

ModuleNotFoundError: No module named 'pandas'

    c. Create one new synthetic feature that could be useful for machine learning in this domain. Explain what it is and
       why it might be useful, and submit code to add it to the dataset.
       
       ANSWER: